In [1]:
%load_ext autoreload
%autoreload 2

try:
    from google.colab import drive
    import os
    drive.mount('/content/drive')
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ConvNetFromScratch'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))
    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)
except:
    pass

import time, os, torch, torchvision, random, time, math
from torch import Tensor
import torchvision
import matplotlib.pyplot as plt
from imageio import imread
from PIL import Image
from torchvision.transforms import ToTensor

%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 10
from toolset.utils import *
from toolset.data import *
from toolset.helper import *
from toolset.solver import *
from convolutional_networks import *
from fully_connected_networks import *
from toolset import *
from typing import Dict, List, Optional
TensorDict = Dict[str, torch.Tensor]
if torch.cuda.is_available:
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Good to go!


In [14]:
N, C, h, w, ph, pw = 10, 3, 5, 5, 2, 2
x = torch.rand(N, C, ph*pw, h*w)
idx = torch.max(x, dim = 2).indices
print(idx.shape)


torch.Size([10, 3, 25])


In [31]:
reset_seed(0)
N, C, h, w, ph, pw = 2, 3, 2, 2, 2, 2
x = torch.rand(N, C, ph*pw, h*w)
print("x shape ", x.shape)
print(x)
print()
_, idx = torch.max(x, dim=2)  # 使用torch.max返回values和indices
print(idx.shape)
print(idx)


x.zero_().scatter_(2, idx.unsqueeze(2), 1)
# output = torch.zeros_like(x)
# print(output.shape)
# output.scatter_(2, idx.unsqueeze(2), 1)
# 观察，最好纸上画个图加深理解

x shape  torch.Size([2, 3, 4, 4])
tensor([[[[0.4963, 0.7682, 0.0885, 0.1320],
          [0.3074, 0.6341, 0.4901, 0.8964],
          [0.4556, 0.6323, 0.3489, 0.4017],
          [0.0223, 0.1689, 0.2939, 0.5185]],

         [[0.6977, 0.8000, 0.1610, 0.2823],
          [0.6816, 0.9152, 0.3971, 0.8742],
          [0.4194, 0.5529, 0.9527, 0.0362],
          [0.1852, 0.3734, 0.3051, 0.9320]],

         [[0.1759, 0.2698, 0.1507, 0.0317],
          [0.2081, 0.9298, 0.7231, 0.7423],
          [0.5263, 0.2437, 0.5846, 0.0332],
          [0.1387, 0.2422, 0.8155, 0.7932]]],


        [[[0.2783, 0.4820, 0.8198, 0.9971],
          [0.6984, 0.5675, 0.8352, 0.2056],
          [0.5932, 0.1123, 0.1535, 0.2417],
          [0.7262, 0.7011, 0.2038, 0.6511]],

         [[0.7745, 0.4369, 0.5191, 0.6159],
          [0.8102, 0.9801, 0.1147, 0.3168],
          [0.6965, 0.9143, 0.9351, 0.9412],
          [0.5995, 0.0652, 0.5460, 0.1872]],

         [[0.0340, 0.9442, 0.8802, 0.0012],
          [0.5936, 0.4158, 0.4

tensor([[[[1., 1., 0., 0.],
          [0., 0., 1., 1.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],

         [[0., 0., 0., 0.],
          [0., 1., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 1.]]],


        [[[0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [0., 0., 0., 0.],
          [1., 1., 0., 0.]],

         [[0., 0., 0., 0.],
          [1., 1., 0., 0.],
          [0., 0., 1., 1.],
          [0., 0., 0., 0.]],

         [[0., 1., 1., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 1.],
          [1., 0., 0., 0.]]]])

In [88]:
class MaxPoolMy(object):

    @staticmethod
    def forward(x:Tensor, pool_param):
        """
        最大池化层前向传播的朴素实现。

        输入：
        - x：输入数据，形状为(N, C, H, W)
        - pool_param：字典，包含以下键：
        - 'pool_height'：每个池化区域的高度
        - 'pool_width'：每个池化区域的宽度
        - 'stride'：相邻池化区域之间的距离
        这里不需要填充。

        返回一个元组：
        - out：形状为(N, C, H', W')的输出，其中H'和W'由以下公式给出：
        H' = 1 + (H - pool_height) / stride
        W' = 1 + (W - pool_width) / stride
        - cache：(x, pool_param)
        """
        N, C, H, W = x.shape
        ph, pw, stride = pool_param['pool_height'], pool_param['pool_width'], pool_param['stride']
        H_out, W_out = 1 + (H - ph) // stride, 1 + (W - pw) // stride
        out = torch.zeros(N, C, H_out, W_out, dtype=x.dtype, device=x.device)

        x_conv = x.unfold(2, ph, stride).unfold(3, pw, stride)  #@ (N,C,H,W)->(N,C,h,w,ph,pw)
        x_colvectorized = x_conv.permute(0, 1, 4, 5, 2, 3).reshape(N, C,ph*pw, -1)   # (N,C,ph,pw,h,w)->(N,C,phpw,hw)
        x_col_value, x_col_idx = x_colvectorized.max(dim = 2)  # (N,C,hw)
        out = x_col_value.reshape(N, C, H_out, W_out)  # (N,C,hw)->(N,C,h,w)
        cache = (x.shape, x_col_idx, x_colvectorized, pool_param)
        return out, cache

    @staticmethod
    def backward(dout:Tensor, cache:Tuple[Tuple,Tensor,Tensor,Dict]):
        """
        最大池化层反向传播的朴素实现。
        输入：
        - dout：上游导数
        - cache：与前向传播中的(x, pool_param)相同的元组。
        返回：
        - dx：相对于x的梯度
        """
        x_shape, x_col_idx, x_colvectorized, pool_param = cache  # x_col_idx : (N,C,hw)
        N, C, H, W = x_shape
        ph, pw, stride = pool_param['pool_height'], pool_param['pool_width'], pool_param['stride']
        H_out, W_out = 1 + (H - ph) // stride, 1 + (W - pw) // stride
        x_colvectorized.zero_().scatter_(2, x_col_idx.unsqueeze(2), 1)  #? why  (N,C,phpw,hw)=(N,C,A,B)
        
        dout_reshape = dout.reshape(N,C,-1)  # (N,C,B)
        tmp = torch.einsum('NCB,NCAB->NCAB',dout_reshape,x_colvectorized).reshape(N, C, ph, pw, H_out, W_out).permute(0, 1, 4, 5, 2, 3)
        dx = torch.zeros(size=x_shape,dtype=x_colvectorized.dtype,device=x_colvectorized.device)  # (N,C,H,W)
        for top in range(ph):
            down = top + H_out * stride
            for left in range(pw):
                right = left + W_out * stride
                dx[:, :, top:down:stride, left:right:stride] += tmp[:, :, :, :, top, left]
        return dx
     

In [89]:

reset_seed(0)
x_shape = torch.tensor((2, 3, 4, 4))
x = torch.linspace(-0.3, 0.4, steps=torch.prod(x_shape), dtype=torch.float64, device='cuda').reshape(*x_shape)
pool_param = {'pool_width': 2, 'pool_height': 2, 'stride': 2}

out, _ = MaxPoolMy.forward(x, pool_param)

correct_out = torch.tensor([[[[-0.26315789, -0.24842105],
                              [-0.20421053, -0.18947368]],
                             [[-0.14526316, -0.13052632],
                              [-0.08631579, -0.07157895]],
                             [[-0.02736842, -0.01263158],
                              [ 0.03157895,  0.04631579]]],
                            [[[ 0.09052632,  0.10526316],
                              [ 0.14947368,  0.16421053]],
                             [[ 0.20842105,  0.22315789],
                              [ 0.26736842,  0.28210526]],
                             [[ 0.32631579,  0.34105263],
                              [ 0.38526316,  0.4       ]]]],
                           dtype=torch.float64, device='cuda')

# Compare your output with ours. Difference should be on the order of e-8.
print('Testing MaxPool.forward function:')
print('difference: ', grad.rel_error(out, correct_out))

Testing MaxPool.forward function:
difference:  5.921052675939009e-09


In [90]:


reset_seed(0)
x = torch.randn(3, 2, 8, 8, dtype=torch.float64, device='cuda')
dout = torch.randn(3, 2, 4, 4, dtype=torch.float64, device='cuda')
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

dx_num = grad.compute_numeric_gradient(lambda x: MaxPoolMy.forward(x, pool_param)[0], x, dout)

out, cache = MaxPoolMy.forward(x, pool_param)
dx = MaxPoolMy.backward(dout, cache)

print('Testing MaxPool.backward function:')
print('dx error: ', grad.rel_error(dx, dx_num))

Testing MaxPool.backward function:
dx error:  6.653155794014975e-10


In [2]:
# Relative errors should be close to 0.0
from convolutional_networks import Conv, MaxPool, FastConv, FastMaxPool


reset_seed(0)
x = torch.randn(40, 3, 32, 32, dtype=torch.float64, device='cuda')
dout = torch.randn(40, 3, 16, 16, dtype=torch.float64, device='cuda')
x_cuda, dout_cuda = x.to('cuda'), dout.to('cuda')
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

t0 = time.perf_counter()
out_naive, cache_naive = MaxPool.forward(x, pool_param)
t1 = time.perf_counter()
out_fast, cache_fast = FastMaxPool.forward(x, pool_param)
t2 = time.perf_counter()
out_fast_cuda, cache_fast_cuda = FastMaxPool.forward(x_cuda, pool_param)
t3 = time.perf_counter()
out_my, cache_my = MaxPoolMy.forward(x, pool_param)
t4 = time.perf_counter()

print('Testing FastMaxPool.forward:')
print('Naive: %fs' % (t1 - t0))
print('Fast: %fs' % (t2 - t1))
print('Fast CUDA: %fs' % (t3 - t2))
print('My: %fs' % (t4 - t3))
print('Speedup: %fx' % ((t1 - t0) / (t2 - t1)))
print('Speedup CUDA: %fx' % ((t1 - t0) / (t3 - t2)))
print('Difference: ', grad.rel_error(out_naive, out_fast))
print('Difference CUDA: ', grad.rel_error(out_naive, out_fast_cuda.to(out_naive.device)))
print('Difference My: ', grad.rel_error(out_my, out_my.to(out_naive.device)))

t0 = time.perf_counter()
dx_naive = MaxPool.backward(dout, cache_naive)
t1 = time.perf_counter()
dx_fast = FastMaxPool.backward(dout, cache_fast)
t2 = time.perf_counter()
dx_fast_cuda = FastMaxPool.backward(dout_cuda, cache_fast_cuda)
t3 = time.perf_counter()
dx_my = MaxPoolMy.backward(dout_cuda, cache_my)
t4 = time.perf_counter()

print('\nTesting FastMaxPool.backward:')
print('Naive: %fs' % (t1 - t0))
print('Fast: %fs' % (t2 - t1))
print('Fast CUDA: %fs' % (t3 - t2))
print('My: %fs' % (t4 - t3))
print('Speedup: %fx' % ((t1 - t0) / (t2 - t1)))
print('Speedup CUDA: %fx' % ((t1 - t0) / (t3 - t2)))
print('dx difference: ', grad.rel_error(dx_naive, dx_fast))
print('dx difference CUDA: ', grad.rel_error(dx_naive, dx_fast_cuda.to(dx_naive.device)))
print('dx difference My: ', grad.rel_error(dx_naive, dx_my.to(dx_naive.device)))

ImportError: cannot import name 'FastConv' from 'convolutional_networks' (D:\DeepLearning\ConvNetFromScratch\convolutional_networks.py)